#Mercari Price Suggestion Challenge

https://www.kaggle.com/c/mercari-price-suggestion-challenge
需要下载数据文件


In [289]:
import numpy as np
import pandas as pd
import math
from time import time
import datetime

def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5
#Source: https://www.kaggle.com/marknagelberg/rmsle-function


In [290]:
from sklearn.impute import SimpleImputer 
import random

def get_data(dir):
    t=time()
    from sklearn.preprocessing import StandardScaler, MinMaxScaler
    from sklearn.model_selection import train_test_split
    train=pd.read_csv(dir+"train.tsv",sep='\t')
    train=train.sample(n=10000)  #内存小,随机选取部分数据
#     x_train,x_test,y_train,y_test=train_test_split(data,data["price"],test_size=0.2,random_state=13)
#     train=x_train.join(x_test,rsuffix="_%s_" %(text_columns[i]))
#     test=y_train.join(y_test,rsuffix="_%s_" %(text_columns[i]))
    train["name"]=train["name"].fillna(" ")+" "+train["category_name"].fillna(" ")+" "+train["brand_name"].fillna(" ")
    train["item_description"]=train["item_description"].fillna(" ")+" "+train["name"].fillna(" ")
    train.drop(['category_name', 'brand_name'], axis=1)
    train,test,y_train,y_test=train_test_split(train,train["price"],test_size=0.2,random_state=13)
  
#     train=pd.read_csv(dir+"train.tsv",sep='\t').fillna(" ")
#     test=pd.read_csv(dir+"test.tsv",sep='\t').fillna(" ")
#     print(train.shape)
#     #print(train.head(3))
#     print(train.describe())
#     print(test.shape)
#     print(test.head(3))
#     print(test.describe())


#     train.loc[train['price'] <=0] = 1
#     test.loc[test['price'] <=0] = 1
#     print(train.describe())
#     print(test.describe())
    print(" {0} 秒完数据读入".format(time() - t))
 
    t=time()

    x_train=train[["item_condition_id","shipping"]]
    x_test=test[["item_condition_id","shipping"]]
    y_train=train["price"]
    y_test=test["price"]
    
    x_train= pd.DataFrame(SimpleImputer ().fit_transform(x_train))
#     y_train= Imputer().fit_transform(y_train)
    x_test= pd.DataFrame(SimpleImputer ().fit_transform(x_test))
#     y_test= Imputer().fit_transform(y_test)    
    t = time()
    print(x_train.shape)
    #print(x_train.head(3))
    
    x_train,x_test=add_tfidf(x_train,x_test,train,test)
#     x_train,x_test=add_word2vec(x_train,x_test,train,test)
    x_train= pd.DataFrame(SimpleImputer ().fit_transform(x_train))
    x_test= pd.DataFrame(SimpleImputer ().fit_transform(x_test))
    x_train = pd.DataFrame(MinMaxScaler().fit_transform(x_train))
    x_test = pd.DataFrame(MinMaxScaler().fit_transform(x_test))

    # print(x_train.head(3))
    print(x_train.shape)
    # print(x_test.head(3))
#     print(x_test.shape)
    print(" {0} 秒完成特征提取".format(time() - t))
    t = time()

    return x_train,y_train,x_test,y_test

In [291]:
from sklearn.impute import SimpleImputer 
def get_data1(dir):
    t=time()
    from sklearn.preprocessing import StandardScaler, MinMaxScaler
    from sklearn.model_selection import train_test_split
    data=pd.read_csv(dir+"train.tsv",sep='\t').fillna(" ")
#     x_train,x_test,y_train,y_test=train_test_split(data,data["price"],test_size=0.2,random_state=13)
#     train=x_train.join(x_test,rsuffix="_%s_" %(text_columns[i]))
#     test=y_train.join(y_test,rsuffix="_%s_" %(text_columns[i]))
    
    train,test,y_train,y_test=train_test_split(data,data["price"],test_size=0.2,random_state=13)
  
#     train=pd.read_csv(dir+"train.tsv",sep='\t').fillna(" ")
#     test=pd.read_csv(dir+"test.tsv",sep='\t').fillna(" ")
#     print(train.shape)
#     #print(train.head(3))
#     print(train.describe())
#     print(test.shape)
#     print(test.head(3))
#     print(test.describe())


#     train.loc[train['price'] <=0] = 1
#     test.loc[test['price'] <=0] = 1
#     print(train.describe())
#     print(test.describe())
    print(" {0} 秒完数据读入".format(time() - t))
 
    t=time()
    #此时归一化，tfidf会自私归一化，而且耗时很长
    x_train=train[["item_condition_id","shipping"]]
    x_test=test[["item_condition_id","shipping"]]
    y_train=train["price"]
    y_test=test["price"]
    
    x_train= pd.DataFrame(SimpleImputer ().fit_transform(x_train))
#     y_train= Imputer().fit_transform(y_train)
    x_test= pd.DataFrame(SimpleImputer ().fit_transform(x_test))
#     y_test= Imputer().fit_transform(y_test)    
    t = time()
    print(x_train.shape)
    #print(x_train.head(3))
    
    x_train,x_test=add_tfidf2(x_train,x_test,train,test)
#     x_train,x_test=add_word2vec(x_train,x_test,train,test)
    x_train= pd.DataFrame(SimpleImputer ().fit_transform(x_train))
    x_test= pd.DataFrame(SimpleImputer ().fit_transform(x_test))
    x_train = pd.DataFrame(MinMaxScaler().fit_transform(x_train))
    x_test = pd.DataFrame(MinMaxScaler().fit_transform(x_test))

    # print(x_train.head(3))
    print(x_train.shape)
    # print(x_test.head(3))
#     print(x_test.shape)
    print(" {0} 秒完成特征提取".format(time() - t))
    t = time()

    return x_train,y_train,x_test,y_test

In [292]:
from sklearn.impute import SimpleImputer 
def get_data2(dir):
    t=time()
    from sklearn.preprocessing import StandardScaler, MinMaxScaler
    train=pd.read_csv(dir+"train.tsv",sep='\t').fillna(" ")
    test=pd.read_csv(dir+"test.tsv",sep='\t').fillna(" ")
    print(train.shape)
    #print(train.head(3))
#     print(train.describe())
    print(test.shape)
    #print(test.head(3))
#     print(test.describe())


#     train.loc[train['price'] <=0] = 1
#     test.loc[test['price'] <=0] = 1
#     print(train.describe())
#     print(test.describe())
    print(" {0} 秒完数据读入".format(time() - t))
 
    t=time()
    #此时归一化，tfidf会自私归一化，而且耗时很长
    x_train=train[["item_condition_id","shipping"]]
    x_test=test[["item_condition_id","shipping"]]
    y_train=train["price"]
    y_test=test["price"]
    
    x_train= pd.DataFrame(SimpleImputer ().fit_transform(x_train))
#     y_train= Imputer().fit_transform(y_train)
    x_test= pd.DataFrame(SimpleImputer ().fit_transform(x_test))
#     y_test= Imputer().fit_transform(y_test)    
    t = time()
    print(x_train.shape)
    #print(x_train.head(3))
    
#     x_train,x_test=add_tfidf(x_train,x_test,train,test)
    x_train,x_test=add_word2vec(x_train,x_test,train,test)
    x_train= pd.DataFrame(SimpleImputer ().fit_transform(x_train))
#     y_train= Imputer().fit_transform(y_train)
    x_test= pd.DataFrame(SimpleImputer ().fit_transform(x_test))
#     y_test= Imputer().fit_transform(y_test)

    x_train = pd.DataFrame(MinMaxScaler().fit_transform(x_train))
    x_test = pd.DataFrame(MinMaxScaler().fit_transform(x_test))

    # print(x_train.head(3))
    print(x_train.shape)
    # print(x_test.head(3))
    print(x_test.shape)
    print(" {0} 秒完成特征提取".format(time() - t))
    t = time()

    return x_train,y_train,x_test,y_test

# pre-trained word vectors from https://nlp.stanford.edu/projects/glove/
# 用法https://medium.com/@japneet121/word-vectorization-using-glove-76919685ee0b
# 安装https://github.com/maciejkula/glove-python/wiki/Installation-on-Windows

def line2vec(line):
    words=str(line).lower().split()
    M=[]
    for word in words:
        try:
            M.append()

def add_glove(x_train,x_test,train,tes):
    text_columns=["category_name","brand_name","name","item_description"]
    max_features=[1000,10000,20000,100000]
    max_features=[100,1000,2000,10000]
    tfidfer=[None]*4
    for i in range(4):
        series=train[text_columns[i]].astype(str)
        print(type(series))
#         print(series.head(2))
        embeddings_index={}
        lines=series.values.tolist()
        for line in lines:
            words=line.split()
            coefs=np.asarray(words[1:],dtype='float64')
            edbeddings_index[word]=coefs
        print('Found %s word vectors.' % len(embeddings_index))
        
        embedding_matrix=np.zeros(len(embeddings_index)) 
#         model=Word2Vec(sentences,size=num_feature)
        b=pd.DataFrame(getAvgFeatureVecs(sentences,model,num_feature))
        
        x_train=x_train.join(b,rsuffix="_%s_" %(text_columns[i]))
        
        series=test[text_columns[i]].astype(str)
        lines=series.values.tolist()
        sentences=[line.split() for line in lines]

        
        d=pd.DataFrame(getAvgFeatureVecs(sentences,model,num_feature))
        x_test=x_test.join(d,rsuffix="_%s_"%(text_columns[i]))

        print(x_train.shape)
#         print(x_train.head(3))
        print(x_test.shape)
#         print(x_test.head(3))
    return x_train,x_test

In [293]:
from sklearn.feature_extraction.text import TfidfVectorizer
def add_tfidf(x_train,x_test,train,test):
# 不可四列一起训练，name band 太稀少了  不划算
    text_columns=["name","item_description"]
    max_features=[100,1000]
    ngrams=[1,1]
    tfidfer=[None]*2
    for i in range(len(tfidfer)):
        tfidfer[i]=TfidfVectorizer(max_features=max_features[i],ngram_range=(1,ngrams[i]))#.fit_transform(train[text_columns[i]])
        a=tfidfer[i].fit_transform(train[text_columns[i]])
        b=pd.DataFrame(a.toarray(),columns=tfidfer[i].get_feature_names())
        # print(tfidfer[i].get_feature_names())
        print(b.shape)
     #   print(b.head(3))
        x_train=x_train.join(b,rsuffix="_%s_" %(text_columns[i]))
        del a,b
        c=tfidfer[i].transform(test[text_columns[i]])
        d=pd.DataFrame(c.toarray(),columns=tfidfer[i].get_feature_names())
        x_test=x_test.join(d,rsuffix="_%s_"%(text_columns[i]))

    return x_train,x_test

In [294]:
from sklearn.feature_extraction.text import TfidfVectorizer
def add_tfidf2(x_train,x_test,train,test):
# 不可四列一起训练，name band 太稀少了  不划算
    text_columns=["category_name","brand_name","name","item_description"]
    max_features=[1000,10000,20000,100000]
    max_features=[100,1000,2000,10000]
    tfidfer=[None]*4
    for i in range(4):
        tfidfer[i]=TfidfVectorizer(max_features=max_features[i],ngram_range=(1,1)).fit(train[text_columns[i]])
        a=tfidfer[i].transform(train[text_columns[i]])
        b=pd.DataFrame(a.toarray(),columns=tfidfer[i].get_feature_names())
        # print(tfidfer[i].get_feature_names())
        print(b.shape)
     #   print(b.head(3))
        x_train=x_train.join(b,rsuffix="_%s_" %(text_columns[i]))

        c=tfidfer[i].transform(test[text_columns[i]])
        d=pd.DataFrame(c.toarray(),columns=tfidfer[i].get_feature_names())
        x_test=x_test.join(d,rsuffix="_%s_"%(text_columns[i]))

    return x_train,x_test

In [295]:
from gensim.models import Word2Vec
def add_word2vec(x_train,x_test,train,test):
# 不可四列一起训练，name band 太稀少了  不划算
    text_columns=["category_name","brand_name","name","item_description"]
    num_features=[50,50,100,100]
    for i in range(4):
        series=train[text_columns[i]].astype(str)
        print(type(series))
#         print(series.head(2))
        lines=series.values.tolist()
        print(type(lines[0]))
#         print(lines[0:3])
#             cutWords_list = [k.split() for k in file.readlines()]
#         sentences=[line.split() for line in lines]
        sentences=lines
        print(type(sentences))
        # 模型参数
        num_feature = num_features[i]    # Word vector dimensionality                      
        min_word_count = 40   # Minimum word count                        
        num_workers = 4       # Number of threads to run in parallel
        context = 10          # Context window size                                                                                    
        downsampling = 1e-3   # Downsample setting for frequent words
        model = Word2Vec(sentences, workers=num_workers, \
                    size=num_feature, min_count=min_word_count, \
                    window=context, sample=downsampling)
#         model=Word2Vec(sentences,size=num_feature)
        b=pd.DataFrame(getAvgFeatureVecs(sentences,model,num_feature))
        
        x_train=x_train.join(b,rsuffix="_%s_" %(text_columns[i]))
        
        series=test[text_columns[i]].astype(str)
        lines=series.values.tolist()
        sentences=[line.split() for line in lines]

        
        d=pd.DataFrame(getAvgFeatureVecs(sentences,model,num_feature))
        x_test=x_test.join(d,rsuffix="_%s_"%(text_columns[i]))

        print(x_train.shape)
#         print(x_train.head(3))
        print(x_test.shape)
#         print(x_test.head(3))
    return x_train,x_test

In [296]:
def makeFeatureVec(sentence, model, num_features):
    featureVec = np.zeros((num_features,), dtype="float64")
    nwords = 0.
    index2word_set = set(model.wv.index2word)
    for word in sentence:
        if word in index2word_set:
            nwords = nwords + 1.
            featureVec = np.add(featureVec, model[word])

    featureVec = np.divide(featureVec, nwords)
    return featureVec

def getAvgFeatureVecs(sentences, model, num_feature):
    counter = 0
    vecs = np.zeros((len(sentences), num_feature), dtype="float64")

    for sentence in sentences:
#         if counter % 10000 == 0:
#             print("sentence %d of %d" % (counter, len(sentences)))

        vecs[counter] = makeFeatureVec(sentence, model, num_feature)
        counter += 1

    return vecs

In [297]:

def ml_predict(gen_models,x_train,y_train,x_test,y_test):

    models=gen_models()
    result=[["model","rmsle"],["time"]]
    for model in models:
        t=time()
        print()
        print(datetime.datetime.now())        
        model=model.fit(x_train,y_train)
        y_predict=model.predict(x_test)
        y_predict[y_predict<=0] = 1
        error=rmsle(y_test,y_predict)
        print(datetime.datetime.now())
        print(model.__module__+"训练结束")
        print("rmsle:%f" %error)
#         print("训练结束"+model.get_params)
        eclapsed=time() - t
        print(" {0} 秒耗时总计".format(eclapsed))
#         result.append([model.__name__,error,eclapsed])
        result.append(["name",error,eclapsed])
        
#         import matplotlib as mpl
#         import matplotlib.pyplot as plt
#         axis=np.arange(len(y_test))
#         mpl.rcParams['font.sans-serif'] = ['simHei']
#         mpl.rcParams['axes.unicode_minus'] = False
#         plt.figure(facecolor='w')
#         plt.plot(t, y_test, 'r-', lw=2, label='真实值')
#         plt.plot(t, y_predict, 'g-', lw=2, label='估计值')
#         plt.legend(loc='best')
# #         plt.title(model.___name_, fontsize=18)
#         plt.xlabel('商品编号', fontsize=15)
#         plt.ylabel('商品价格', fontsize=15)
#         plt.grid()
#         plt.show()

In [298]:

def dl_predict(model,x_train,y_train,x_test,y_test):

    result=[["model","rmsle"],["time"]]

    t=time()
    print()
    print(datetime.datetime.now())        
    y_predict=model(x_train,y_train,x_test)
    y_predict[y_predict<=0] = 1
    error=rmsle(y_test,y_predict)
    print(datetime.datetime.now())
    print(model.__module__+"训练结束")
    print("rmsle:%f" %error)
#         print("训练结束"+model.get_params)
    eclapsed=time() - t
    print(" {0} 秒耗时总计".format(eclapsed))
#         result.append([model.__name__,error,eclapsed])
    result.append(["name",error,eclapsed])

#         import matplotlib as mpl
#         import matplotlib.pyplot as plt
#         axis=np.arange(len(y_test))
#         mpl.rcParams['font.sans-serif'] = ['simHei']
#         mpl.rcParams['axes.unicode_minus'] = False
#         plt.figure(facecolor='w')
#         plt.plot(t, y_test, 'r-', lw=2, label='真实值')
#         plt.plot(t, y_predict, 'g-', lw=2, label='估计值')
#         plt.legend(loc='best')
# #         plt.title(model.___name_, fontsize=18)
#         plt.xlabel('商品编号', fontsize=15)
#         plt.ylabel('商品价格', fontsize=15)
#         plt.grid()
#         plt.show()



模型








In [299]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso, RidgeCV, LassoCV, ElasticNetCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
def gen_linear():
    linear=LinearRegression()
    ridge=Ridge()
    lasso=Lasso()
#     linear_pipeline=Pipeline([
#             ('poly', PolynomialFeatures()),
#             ('linear', ElasticNetCV(alphas=np.logspace(-3, 2, 10), l1_ratio=[.1, .5, .7, .9, .95, .99, 1],
#                                     fit_intercept=False))])
#     return [linear,ridge,lasso,linear_pipeline]
    return [linear,ridge,lasso]


In [300]:
from sklearn.tree import DecisionTreeRegressor
def gen_tree():
    dtr=DecisionTreeRegressor(criterion='mse', max_depth=5)
    return [dtr]

In [301]:
from sklearn.ensemble import BaggingRegressor,RandomForestRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import RidgeCV
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

def gen_ensem():
    ridge = RidgeCV(alphas=np.logspace(-3, 2, 3), fit_intercept=False)
    ridged = Pipeline([('poly', PolynomialFeatures(degree=2)), ('Ridge', ridge)])
    bagr= BaggingRegressor(ridged, n_estimators=10, max_samples=0.3)
    rfr = RandomForestRegressor(n_estimators=10, criterion='entropy', max_depth=5, oob_score=True)
    dbr= GradientBoostingRegressor(n_estimators=10, learning_rate=0.1, max_depth=2)
    base_estimator = DecisionTreeRegressor(criterion='gini', max_depth=3, min_samples_split=4)
    adar = AdaBoostRegressor(base_estimator=base_estimator, n_estimators=10, learning_rate=0.1)

    params={"max_depth":3,"learning_rate":0.1,"n_estimators":10,"silent":0,"objective":"reg:linear"}
    xgbr=xgb.XGBRegressor(**params)
    model=xgb.XGBRegressor()
    parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}
    gsxgbr = GridSearchCV(model,parameters,cv = 5,n_jobs = 5,verbose=True)
    return [bagr,rfr,dbr,adar,xgbr,gsxgbr]

In [302]:
from sklearn import svm
def gen_svm():
    svr_linear = svm.SVR(kernel='linear', C=100)
    svr_poly = svm.SVR(kernel='poly', degree=3, C=100)
    svr_rbf = svm.SVR(kernel='rbf', gamma=0.2, C=100)
    return [svr_linear,svr_rbf,svr_poly]

In [303]:
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture
def gen_em():
    gmm = GaussianMixture(n_components=1, covariance_type='full', random_state=0)
    dpgmm = BayesianGaussianMixture(n_components=1, covariance_type='full', max_iter=1000, n_init=5,
                                    weight_concentration_prior_type='dirichlet_process', weight_concentration_prior=0.1)
    return [gmm,dpgmm]

In [304]:
import os; os.environ['OMP_NUM_THREADS'] = '4'
import torch
import torch.nn as nn

def df2tc(df):
    features=torch.FloatTensor(df.values)
    tc=torch.unsqueeze(features,dim=1)
    tc=torch.autograd.Variable(tc)
    return tc
    
def tc2df(tc):
    n=tc.data.numpy()
    return pd.DataFrame(n)
    
def fcnn(x_train,y_train,x_test):
    print("开始构建神经网络")

    # Device configuration
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model_path="fcnn.model"
    # Hyper-parameters
    input_size = len(x_train.iloc[0])
    num_epochs = 100
    learning_rate = 0.1

#     x_train=torch.from_numpy(x_train.values)
    
#     y_train=torch.tensor(y_train)
#     x_test=torch.tensor(x_test)
    x_train_features=torch.FloatTensor(x_train.values)
    x_train=torch.unsqueeze(x_train_features,dim=1)
    x_train=torch.autograd.Variable(x_train)
#     x_train,y_train,x_test=torch.autograd.Variable(x_train,y_train,x_test)
#     x_train=torch.from_numpy(x_train.values.astype(np.float32))
#     input_size = x_train.size(0)


    # Fully connected neural network with one hidden layer
    class NeuralNet(nn.Module):
        def __init__(self,input_size):
            super(NeuralNet, self).__init__()
            self.model = nn.Sequential(
                                        nn.Linear(input_size, 1024),
                                        nn.ReLU(),                                        
                                        nn.Linear(1024, 64),
                                        nn.ReLU(),
                                        nn.Linear(1024, 64),
                                        nn.ReLU(),
                                        nn.Linear(64, 1))

        def forward(self, x):
            # out = self.fc1(x)
            # out = self.relu(out)
            # out = self.fc2(out)
            # return out
            return self.model(x)
        
    net = NeuralNet(input_size).to(device)
    print('神经网络为 {}'.format(net))

    # Loss and optimizer
    loss_func = torch.nn.MSELoss()      # 预测值和真实值的误差计算公式 (均方差)
    optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate)  # 传入 net 的所有参数, 学习率

    if not os.path.exists(model_path):
        # Train the model
        for epoch in range(num_epochs):
            out = net(x_train)  # 喂给 net 训练数据 x, 每迭代一步，输出预测值
            loss = loss_func(out, y_train)  # 计算两者的误差
            optimizer.zero_grad()   # 清空上一步的残余更新参数值
            loss.backward()         # 误差反向传播, 计算参数更新值
            optimizer.step()        # 将参数更新值施加到 net 的 parameters 上
            if epoch % 1 == 0:
                print('Epoch [{}/{}]  Loss: {:.4f}'  .format(epoch , num_epochs, loss.item()))
                print(datetime.datetime.now())

                # Save the model checkpoint
        torch.save(model.state_dict(), model_path)
    else:
        net.load_state_dict(torch.load(model_path))
        net.eval()

    #模型預測
    y_predicted = net(torch.from_numpy(x_test)).detach().numpy()
    y_predicted=y_predicted.data.numpy()
    y_predicted=pd.DataFrame(y_predicted)
    return y_predicted


In [305]:
if __name__=="__main__":
    t0 = time()
    t = t0
    print("开始时间: ")
    print(datetime.datetime.now())
    dir = r"input/"
    x_train,y_train,x_test,y_test=get_data(dir)

    gens=[gen_linear,gen_tree,gen_svm,gen_ensem,gen_em]
#     gens=[gen_linear]
#     for gen in gens:
#         ml_predict(gen,x_train,y_train,x_test,y_test)
    deeps=[fcnn]
    for model in deeps:
        dl_predict(model,x_train,y_train,x_test,y_test)

    print(" 所有模型预测完毕  {0} 秒耗时总计".format(time() - t))

开始时间: 
2019-03-03 21:14:36.984825
 5.190158843994141 秒完数据读入
(8000, 2)
(8000, 100)
(8000, 1000)
(8000, 1102)
 1.2000365257263184 秒完成特征提取

2019-03-03 21:14:43.422018
开始构建神经网络
神经网络为 NeuralNet(
  (model): Sequential(
    (0): Linear(in_features=1102, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1024, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=1, bias=True)
  )
)


RuntimeError: size mismatch, m1: [8000 x 64], m2: [1024 x 64] at c:\a\w\1\s\tmp_conda_3.6_090826\conda\conda-bld\pytorch_1550394668685\work\aten\src\th\generic/THTensorMath.cpp:940

In [ ]:

    t0 = time()
    t = t0
    print("开始时间: ")
    print(datetime.datetime.now())
    dir = r"input/"
    x_train,y_train,x_test,y_test=get_data(dir)


In [ ]:
    deeps=[fcnn]
    for model in deeps:
        dl_predict(model,x_train,y_train,x_test,y_test)